# Primary Notebook

## Purpose

The purpose of this notebook is the creation of the clusters.
Those (landscape) clusters are supposed to be areas similar in appearance;
To be more precise, these clusters share a common "FFT footprint" which will be elaborated on later.

## Other parts of this project

There are secondary notebooks, that might have to be run beforehand, to receive data which this primary notebook will work upon.

In [1]:
# What we might need eventually
# rasterio for geotiffs
# dask array for parallel computing of large arrays
# pyfftw for 2d fft 

## Overview of the classes and their interconnection
The subject of this notebook is to cluster different landscapes across multiple DEM-GeoTIFFs. 

### GeographicBounds
This is an object that saves West, South, East, North borders, as well as the projection, and maybe some other additional info.

### AugmentedDEM
This will be the "container" for all information regarding *one* specific DEM raster map.

### EmbeddingMap
This will be part of each AugmentedDEM. Here we will save the labels created by the clustering.




## How to make the GeoTIFF images usable by my algorithm:
It’s important to note, that the original DEM data is in arc seconds. To avoid skewed results it has to be compressed by cos(latitude) in width.  That way the metric distances (almost) resemble the distances pixel-wise

After that the images will be scaled to 1/4 (as of now), because processing becomes about 4^2 times faster.

Before processing, the data will have to be split up into quadratic tiles which are supposed to be equal in their length.

I guess we should combine the processes, namely selecting tiles of equal size and then reducing their size to a given size in pixels

In [2]:
# Imports

# Filesystem, JSON
import os
import json
from io import BytesIO # To be able to route the API response to a file-like object that rasterio can use

# Rasterio for handling GeoTIFFs
import rasterio
from rasterio.transform import Affine
from rasterio.windows import from_bounds

# For interpolation of the results
from scipy.interpolate import griddata

# Math may not be missing
import math
import numpy as np
import dask.array as da
import pyfftw 

# The heart: we use k-means-clustering
from sklearn.cluster import KMeans

# Regex for updating filenames
import re

# For earth related numbers
from pyproj import Geod
geod = Geod(ellps="WGS84")

# Matplotlib for checks
import matplotlib.pyplot as plt
from matplotlib import colors

# For timing 
import time


In [3]:
class SimpleTimer:
    '''This class is just a timer for checking the performance'''
    def __init__(self, description):
        self.description = description
    
    def __enter__(self):
        self.timer = time.perf_counter()
        return self

    def __exit__(self, type, value, traceback):
        self.time_needed = time.perf_counter() - self.timer
        print(f"{self.description} took {self.time_needed:.1f} Seconds")
    

In [4]:
# Settings for running this Notebook

internal_settings = {}

internal_settings["files"] = {}
internal_settings["files"]["dem_folder"] = "geotiffs" #Here the to-be-used-geotiffs are located

internal_settings["fft"] = {}
internal_settings["fft"]["tile_size_km"] = 10 #average length and width of a tile that is processed individually
internal_settings["fft"]["tile_size_px"] = 15

internal_settings["fft"]["tile_overlap_percent"] = 50 # How much (by a tile length) do the tiles overlap
internal_settings["fft"]["tile_overlap_multi"] =  1 / (1 - (internal_settings["fft"]["tile_overlap_percent"]/100))
internal_settings["fft"]["fft_levels"] = 15  

internal_settings["output"] = {}
internal_settings["output"]["q_factor"] = 10


In [5]:
# Define classes specific to this project 

# This class is supposed to store coordinates for tiles and maybe coordinate systems in the future
class GeographicBounds:
    def __init__(self, west, south, east, north):
        self.west = west
        self.north = north
        self.east = east
        self.south = south

    @property
    def center_x (self): # Center longitude
        return (self.west+self.east)/2

    @property
    def center_y (self): # Center latitude
        return (self.north+self.south)/2
    
    @property
    def center_combined (self): # Center latitude
        return (self.center_y, self.center_x)

    @property
    def height_degrees (self): # Height in kilometres
        return (abs(self.south - self.north))

    @property
    def width_degrees (self): # Height in kilometres
        return min (abs(self.east - self.west), 360 - abs(self.east - self.west));  #when getting to ±180° it should choose the shorter width # CAUTION, this might be not the final solution

    @property
    def height_km (self): # Height in kilometres
        return (abs(self.south - self.north) / 360.0) * math.pi * 2.0 * geod.b / 1000.0
   
    @property
    def width_north_km (self): # North width in kilometres
        return (abs(self.west - self.east) / 360.0) * math.pi * 2.0 * geod.a * math.cos( math.radians(self.north) ) / 1000.0 
   
    @property
    def width_south_km (self): # North width in kilometres
        return (abs(self.west - self.east) / 360.0) * math.pi * 2.0 * geod.a * math.cos( math.radians(self.south) ) / 1000.0 

    # Return information about the object (for to be used in a print statement for example)
    def __str__(self):
        infostring = "Infos about the GeographicBounds (all rounded):\n"
        infostring += f"-----\n"
        infostring += f"Geographic Bounds:\n"
        infostring += f"West: {self.west:>8.2f}° \n"
        infostring += f"South: {self.south:>7.2f}° \n"
        infostring += f"East: {self.east:>8.2f}° \n"
        infostring += f"North: {self.north:>7.2f}\n"
        infostring += f"-----\n"
        infostring += f"Northern Width: {self.width_north_km:.2f} km\n"
        infostring += f"Southern Width: {self.width_south_km:.2f} km\n"
        infostring += f"Height: {self.height_km:.2f} km\n"
        infostring += f"-----\n"
        infostring += f"Center (lon E, lat N): {self.center_y:.3f}°, {self.center_x:.3f}°\n"
        infostring += "\n"
        return infostring
    
    def as_list(self):
        return [self.west, self.south, self.east, self.north]


In [ ]:
    
# This class will contain information about each geotiff that is used for the fft calculations
class AugmentedDEM:
    def __init__(self, dem_path):
        with rasterio.open(dem_path) as dem_file:
            self.geo_bounds = GeographicBounds(dem_file.bounds.left, 
                                               dem_file.bounds.bottom,
                                               dem_file.bounds.right,
                                               dem_file.bounds.top)
            self.dem_width_px = dem_file.width
            self.dem_height_px = dem_file.height
        self.dem_path = dem_path
        


    def CalculateTileBounds(self): # alternative implementation that gives back the borders and stuff in degrees
        # Height of a tile in degrees
        
        # Squeeze more tiles in
        tilemulti = internal_settings["fft"]["tile_overlap_multi"]

        tile_height_degrees = 360 * internal_settings["fft"]["tile_size_km"] * 1000 /  (math.pi * 2.0 * geod.b)

        relative_start_y = (self.geo_bounds.height_degrees % tile_height_degrees) / 2
        absolute_start_y = (self.geo_bounds.north - relative_start_y)
        absolute_end_y = (self.geo_bounds.south + relative_start_y)
        number_tiles_y = int((self.geo_bounds.height_degrees // tile_height_degrees)*tilemulti)

        absolute_borders_y = np.linspace(absolute_start_y, absolute_end_y, number_tiles_y + 1 )
        absolute_centers_y = np.linspace(absolute_start_y - (tile_height_degrees / 2), absolute_end_y + (tile_height_degrees / 2), number_tiles_y )


        # Here will go the X-splitting. X will be split with a different tile_width_degrees for each centers_y
        # That is due to the fact that x inside a sector of the earth’s surface is getting smaller in direction of the poles (cos)
        # This is why this is a list of np.arrays and not a 2D np-array (as length varies for each row!)

        absolute_borders_x = []
        self.tile_bounds = []

        
        for i, center_y in enumerate(absolute_centers_y):
            tile_width_degrees = 360 * internal_settings["fft"]["tile_size_km"] * 1000 /  (math.pi * 2.0 * geod.a)  / math.cos (math.radians(center_y))
            
            relative_start_x = (self.geo_bounds.width_degrees % tile_width_degrees) / 2
            absolute_start_x = (self.geo_bounds.west + relative_start_x)
            absolute_end_x = (self.geo_bounds.east - relative_start_x)
            number_tiles_x = int((self.geo_bounds.width_degrees // tile_width_degrees)*tilemulti)

            tmp_absolute_borders_x = np.linspace(absolute_start_x, absolute_end_x, number_tiles_x + 1 )
            tmp_absolute_centers_x = np.linspace(absolute_start_x + (tile_width_degrees / 2), absolute_end_x - (tile_width_degrees / 2), number_tiles_x)

            absolute_borders_x.append( np.array(tmp_absolute_borders_x) )

        
        for y in range(len(absolute_centers_y-1)):
            for x in range(len(absolute_borders_x[y])-1):
                self.tile_bounds.append(
                    GeographicBounds(absolute_borders_x[y][x],
                                    absolute_borders_y[y],
                                    absolute_borders_x[y][x+1],
                                    absolute_borders_y[y+1]))
                
    def ResampleTiles(self):
        '''This method resamples tiles from the original dem data
        The tiles all have the same pixel dimensions'''


        with SimpleTimer("Resampling the Tiles"):
            with rasterio.open(self.dem_path) as source:
                tile_bounds_np = np.array([bnd.as_list() for bnd in self.tile_bounds])
                num_tiles = len(tile_bounds_np)
                self.tiles_resampled = np.empty((num_tiles, 
                                internal_settings["fft"]["tile_size_px"], 
                                internal_settings["fft"]["tile_size_px"]), dtype = source.dtypes[0])

                for i, bound in enumerate(tile_bounds_np):
                    src_window = from_bounds(bound[0], bound[3], bound[2], bound[1], source.transform) 
                    tile_resampled = source.read(
                        out_shape = (source.count, 
                                    internal_settings["fft"]["tile_size_px"], 
                                    internal_settings["fft"]["tile_size_px"]),
                                    window = src_window,
                                    resampling = rasterio.enums.Resampling.cubic
                    )

                    # Remove erroneous values
                    self.tiles_resampled = np.nan_to_num(self.tiles_resampled, nan = 0)

                    # Remove height from the landscape by substracting the median
                    self.tiles_resampled = self.tiles_resampled - np.mean(self.tiles_resampled, axis = (1,2), keepdims=True)

                    self.tiles_resampled[i] = tile_resampled

    
    
    def DumpTiles(self):
        for i, tile in enumerate(self.tiles_resampled):
            height = internal_settings["fft"]["tile_size_px"]
            width = internal_settings["fft"]["tile_size_px"]

            # Calculate pixel resolution (degrees per pixel)
            res_x = ((self.tile_bounds[i].east - self.tile_bounds[i].west) / width)
            res_y = ((self.tile_bounds[i].north - self.tile_bounds[i].south) / height)

            # Create affine transform (pixel coordinates → WGS84)
            transform = Affine.translation(self.tile_bounds[i].west, self.tile_bounds[i].south) * Affine.scale(res_x, res_y)

            filepath = f"tile_dump/{self.tile_bounds[i].west:.3f}_{self.tile_bounds[i].north:.3f}.tif"

            # Write GeoTIFF
            with rasterio.open(
                filepath,
                "w",
                driver="GTiff",
                height=height,
                width=width,
                count=1, 
                dtype=tile.dtype,
                crs="EPSG:4326", 
                transform=transform,
            ) as dst:
                dst.write(tile[np.newaxis,:])



    def CreateFFTFootprint(self):
        with SimpleTimer("Creating FFT footprints"):
            fft_input_array = pyfftw.empty_aligned((len(self.tiles_resampled),
                                                    internal_settings["fft"]["tile_size_px"],
                                                    internal_settings["fft"]["tile_size_px"]),
                                                    dtype = "complex64")

            fft_output_array =  pyfftw.empty_aligned((len(self.tiles_resampled),
                                                    internal_settings["fft"]["tile_size_px"],
                                                    internal_settings["fft"]["tile_size_px"]),
                                                    dtype = "complex64")

            fft_execution_plan = pyfftw.FFTW(fft_input_array,
                                            fft_output_array, axes=(1,2),
                                            direction="FFTW_FORWARD",
                                            flags=("FFTW_MEASURE",))

            fft_input_array[:] = self.tiles_resampled #Important. [:] ensures the reserved empty array is used, and no new one is created! It will not work without that special slicing.

            fft_execution_plan.execute()

            fft_magnitude_spectra_unshifted = np.log(np.abs(fft_output_array) + 1) 
            self.fft_footprint = np.fft.fftshift(fft_magnitude_spectra_unshifted, axes=(1,2)) #Centered FFT



    def CreateFFTAverages(self):
        # Set up Dask Arrays for faster computation of the weighted averages


        # The general dimensions are:
        # 1. Number of tiles (total)
        # 2. Height of a tile
        # 3. Width of a tile
        # 4. Number of levels (circle filters)

        # To multiply and divide all the dimensions have to be in the same order for all arrays

        # Weights – here go the circles
        # Weights are still (#4, #2, #3), change them to #2, #3, #4


        with SimpleTimer("Creating the weighted averages per mask"):
            da_weights = da.from_array(
                # The height becomes axis 0, width becomes axis 1, and different masks become axis 2
                np.transpose(temporary_settings["circle_masks"],(1,2,0)) 
            )

            # First dimension (#1) is missing, add it 
            # Just a note: the tiles themselves are not x-y adressed, but the get a continuous index (1D)
            da_weights = da_weights[np.newaxis, ...]
            # Now the form of the weights is (#1, #2, #3, #4) like explained above

            # Take the results of the fft and place them also in a dask array
            # of the wanted form (see above) 
            da_magnitude_spectra = da.from_array(self.fft_footprint)[...,np.newaxis]

            # Sum the values of the areas covered by the circles, to later have something to divide by
            # to generate weighted sums
            da_weights_sum = da.sum(da_weights, axis=(1,2)) 

            # Sum the results of the FFTs multiplied by the weights of the different levels (circle filters)
            da_sum_of_spectra = da.sum(da_magnitude_spectra * da_weights, axis=(1, 2))

            # Calculate the weighted average 
            da_weighted_averages = da_sum_of_spectra / da_weights_sum

            # The result is now in the shape of #1, #4 -> Number of Tiles, Number of Levels
            self.fft_magnitude_all_levels = da_weighted_averages.compute() 



    def CreateImage(self):

        # Next up: We now have the results for all the tiles
        # We will need the "position" of all the tiles
        # Then we can interpolate 


        tmp_tile_positions = [tile.center_combined for tile in self.tile_bounds]

        tmp_pic_width = self.dem_width_px // internal_settings["output"]["q_factor"]
        tmp_pic_height = self.dem_height_px // internal_settings["output"]["q_factor"]

        tmp_grid_res_y = np.linspace(self.geo_bounds.north, self.geo_bounds.south, tmp_pic_height )
        tmp_grid_res_x = np.linspace(self.geo_bounds.west, self.geo_bounds.east, tmp_pic_width )

        tmp_grid_y, tmp_grid_x = np.meshgrid(tmp_grid_res_y, tmp_grid_res_x, indexing="ij")

        tmp_interpolated = np.zeros((internal_settings["fft"]["fft_levels"], tmp_pic_height, tmp_pic_width))
        print(tmp_interpolated.shape)

        for i in range(internal_settings["fft"]["fft_levels"]):
            tmp_interpolated[i] = griddata(tmp_tile_positions,
            self.fft_magnitude_all_levels[:, i], 
            (tmp_grid_y, tmp_grid_x), method='cubic')


        # For clustering we remove the picture’s y and x dimensions, as we don’t account for spatial proximity
        tmp_reshaped_result = tmp_interpolated.reshape(internal_settings["fft"]["fft_levels"], -1).T
        
        #Get rid of errors (div/0 and so on)
        tmp_reshaped_result = np.nan_to_num(tmp_reshaped_result, nan=0)

        # AS OF NOW
        n_clusters = 8

        k_means_clusterer = KMeans(n_clusters = n_clusters, random_state=550)

        # Scale the result

        # Standardize
        tmp_reshaped_result = tmp_reshaped_result - np.mean(tmp_reshaped_result, axis = 0)
        tmp_reshaped_result = tmp_reshaped_result / np.std(tmp_reshaped_result, axis = 0)

        #Get rid of errors (div/0 and so on)
        tmp_reshaped_result = np.nan_to_num(tmp_reshaped_result, nan=0)

        with SimpleTimer("Clustering of the results"):
            tmp_labels = k_means_clusterer.fit_predict(tmp_reshaped_result)

        tmp_labels_image = tmp_labels.reshape(tmp_interpolated.shape[1],tmp_interpolated.shape[2])

        # Calculate pixel resolution (degrees per pixel)
        res_x = ((self.geo_bounds.east - self.geo_bounds.west) / tmp_pic_width)
        res_y = ((self.geo_bounds.south - self.geo_bounds.north) / tmp_pic_height)

        # Create affine transform (pixel coordinates → WGS84)
        transform = Affine.translation(self.geo_bounds.west,
                                    self.geo_bounds.north) * Affine.scale(res_x, res_y)


        cmap = plt.get_cmap("viridis")

        tmp_labels_image_sqeezed = np.squeeze(tmp_labels_image)
        normalizer = colors.Normalize(vmin=tmp_labels_image_sqeezed.min(), vmax=tmp_labels_image_sqeezed.max())
        tmp_labels_image_sqeezed_normalized = normalizer(tmp_labels_image_sqeezed)


        tmp_labels_image_rgb = cmap(np.squeeze(tmp_labels_image_sqeezed_normalized))

        tmp_labels_image_rgb = (np.transpose(tmp_labels_image_rgb[...,:3], (2,0,1))*256).astype("uint8")

        filename_string = f"test_result " + \
        f"tlszkm {internal_settings["fft"]["tile_size_km"]:.1f}  " + \
        f"tlszpx {internal_settings["fft"]["tile_size_px"]:.0f}  " + \
        f"fftlvls {internal_settings["fft"]["fft_levels"]:.0f}  " + \
        f"qfctr {internal_settings["output"]["q_factor"]:.0f}  " + \
        f"{internal_settings["fft"]["tile_overlap_multi"]:.1f}x"+ \
        f"-ovrlp-pct-{internal_settings["fft"]["tile_overlap_percent"]:.0f}"+ \
        f".tif"

        # Write GeoTIFF
        with rasterio.open(
            filename_string,
            "w",
            driver="GTiff",
            height=tmp_pic_height,
            width=tmp_pic_width,
            count=3, 
            dtype="uint8",
            crs="EPSG:4326", 
            transform=transform,
        ) as dst:
            dst.write(tmp_labels_image_rgb[0],1)
            dst.write(tmp_labels_image_rgb[1],2)
            dst.write(tmp_labels_image_rgb[2],3)
        

In [7]:
def Euclidean (y_a, y_b, x_a, x_b):
    return math.sqrt((y_a-y_b)**2 + (x_a-x_b)**2)

def Threshold (input, threshold, bandwidth = 1):
    # This ist just for anti aliasing the circle mask
    # The input is claped to the bandwidth
    result = np.interp(input, [threshold-(bandwidth/2), threshold+(bandwidth/2)], [0,1])
    return result;


def CircleImage (height, width, radius, inverted = False, bandwidth = 1):
    # Returns the image of a circle as a numpy array for masking
    # The bandwidth defines the width of the border of the circle (for anti aliasing)

    circle_image = np.zeros((height, width))

    if radius == 0:
        return (1 - circle_image) if inverted else (circle_image)
    else:
        for x in range(width):
            for y in range(height):
                circle_image[y,x] = Euclidean(
                    y+(0.5 if height%2 == 1 else 0), height / 2,
                    x+(0.5 if width%2 == 1 else 0), width / 2
                )

    if inverted:
        return 1-Threshold(circle_image, radius, 1)
    else:
        return Threshold(circle_image, radius, 1)

def RingImage (height, width, inner_radius, outer_radius, bandwidth):
    if outer_radius < inner_radius:
        raise ValueError("The inner radius must be smaller than the outer radius.")

    # This combines two circles to form a ring mask
    outercircle = CircleImage(height, width, outer_radius, 
                              inverted = True, bandwidth=bandwidth)
    innercircle = CircleImage(height, width, inner_radius, 
                              inverted = True, bandwidth=bandwidth)
    
    ringimg = outercircle - innercircle

    return  (ringimg - ringimg.min()) / (ringimg.max() - ringimg.min())

def RingImageSeries(height,width,steps,bandwidth):
    '''This creates a 3D numpy array of the shape
    (Masks, individual Height, individual Width)
    This is used to sum and average the FFT magnitudes'''

    with SimpleTimer("Creating the ring masks"):
        # The diameter gets bigger logarithmically, starting with a diameter of 1
        smallest_side = min(height,width)

        outer_radii = np.logspace(0, np.log2(smallest_side / 2), steps, base=2) # 0 means it starts with 1 (log)
        inner_radii = np.append(0,outer_radii[:-1])


        all_masks = np.zeros((steps,height, width))

        for i in range(steps):
            all_masks[i] = RingImage(height,width,inner_radii[i],outer_radii[i],bandwidth=bandwidth)
            
    return all_masks


In [8]:

temporary_settings = {}
temporary_settings["augmented_dems"] = []

# Put all the DEM maps from the dem_folder into the dem_files
with os.scandir(internal_settings["files"]["dem_folder"]) as dirlist:
    temporary_settings["augmented_dems"] =[AugmentedDEM(d.path) for d in dirlist if re.search(r"\.tif",d.name)]

# Set up the masks for the weighted average
temporary_settings["circle_masks"] = RingImageSeries(internal_settings["fft"]["tile_size_px"],
                                                     internal_settings["fft"]["tile_size_px"],
                                                     internal_settings["fft"]["fft_levels"]
                                                     ,bandwidth = 1)


Creating the ring masks took 0.0 Seconds


In [9]:
# %%timeit -n 10 -r 10
# Testing
# Important! At the moment all the labels are created INSIDE one DEM
# Later they should be done across all dems! 
# That will be what makes them comparable


for d in temporary_settings["augmented_dems"]:
    d.CalculateTileBounds()
    d.ResampleTiles()
    d.CreateFFTFootprint()
    d.CreateFFTAverages()
    with SimpleTimer("Creating the image"):
        d.CreateImage()



Resampling the Tiles took 25.8 Seconds
Creating FFT footprints took 0.0 Seconds
Creating the weighted averages per mask took 0.2 Seconds
(15, 504, 792)
Clustering of the results took 0.4 Seconds
Creating the image took 2.3 Seconds
